# Preprocesamiento y Normalizacion de los datos

- Normalizacion entre 0 y 1 = [Dias_Ultima_Camp,No_Contactos,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,Edad]
- one-hot encoding = [mora,Vivienda,Consumo,Resultado_Anterior,Tipo_Trabajo,Estado_Civil,Educacion,Contacto]

In [1]:
# Importacion de recusos generales
import pandas as pd
import sqlite3
from config.parametros import RutaDataSet as RT
# Recursos propios
from services.transformaciones import CargaDatos
# Preprocesamiento
from sklearn.preprocessing import MinMaxScaler

In [2]:
# Instancia de conexion a la base de datos SQL
conn = sqlite3.connect(RT.DataBase)
cursor = conn.cursor()

In [3]:
# Join de la tabla Qualitative y Campanas
query = """
    SELECT
        c.ID,
        c.mora,
        c.Vivienda,
        c.Consumo,
        c.Mes,
        c.Dias_Ultima_Camp,
        c.No_Contactos,
        c.Resultado_Anterior,
        c.emp_var_rate,
        c.cons_price_idx,
        c.cons_conf_idx,
        c.euribor3m,
        c.nr_employed,
        c.y,
        q.Edad,
        q.Tipo_Trabajo,
        q.Estado_Civil,
        q.Educacion,
        q.Contacto
    FROM Campanas AS c
    JOIN Qualitative AS q ON c.ID = q.ID
    WHERE y IS NOT NULL
"""
df = pd.read_sql_query(query,conn)
# Nota para el desarrollo de modelo predictivo se descarta los valores en blanco de la columna predictora (y)

In [4]:
# Generar una copia de dataset 
df_original = df.copy()
df_original.shape

(16170, 19)

In [5]:
# Definicion de variables
list_min_max = [
    'Dias_Ultima_Camp',
    'No_Contactos',
    'emp_var_rate',
    'cons_price_idx',
    'cons_conf_idx',
    'euribor3m',
    'nr_employed',
    'Edad'
]
list_get_dummies = [
    'mora',
    'Vivienda',
    'Consumo',
    'Mes',
    'Resultado_Anterior',
    'Tipo_Trabajo',
    'Estado_Civil',
    'Educacion',
    'Contacto'
]

In [6]:
# Normalizacion de Dataset en columnas especificadas en list_min_max
scaler = MinMaxScaler()
df[list_min_max] = scaler.fit_transform(df[list_min_max])

In [7]:
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
# Generar columnas usando metodo one-hot encoding a columnas detalladas en list_get_dummies
df_matriz = pd.get_dummies(df,columns=list_get_dummies)
col_bool = df_matriz.iloc[:,-50:] 
df_matriz.iloc[:,-50:] = col_bool.astype(int)
df_matriz['mora_si'] = 0

In [8]:
# Defincion de variables (x) y (y)
X_matriz = df_matriz.drop(columns=['ID','y'],axis=1)
y_matriz = df_matriz['y']
e_matriz = df_matriz['ID']
# Generarar Matricez vectoriales
X = X_matriz.to_numpy() # matriz con las variables independientes
y = y_matriz.to_numpy() # Matriz con la variables dependiente
e = e_matriz.to_numpy() # Vector on los ID de cliente

In [9]:
# Guardar Matriz para modelado
M = CargaDatos(carpeta='Archivos')
M.guardar_matriz_numpy(nombre_matriz='X_CDT',vectores=X)
M.guardar_matriz_numpy(nombre_matriz='y_CDT',vectores=y)
M.guardar_matriz_numpy(nombre_matriz='Etiqueta_cliente',vectores=e)

Matriz guardada exitosamente como X_CDT.npy
Matriz guardada exitosamente como y_CDT.npy
Matriz guardada exitosamente como Etiqueta_cliente.npy
